# Using CleanRL PPO to try and train Villagers

In [1]:
import numpy as np
import torch
import sys
sys.path.append('../')
from voting_games.werewolf_env_v0 import raw_env
import random
from tqdm import tqdm

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
env = raw_env(num_agents=10, werewolves=2)
env.reset()

In [19]:
env.action_spaces['player_1'].sample()

6

In [24]:
env.observation_spaces['player_1'].sample()

OrderedDict([('action_mask',
              array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
                      True])),
             ('observation',
              OrderedDict([('day', 4),
                           ('phase', 0),
                           ('player_status',
                            array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
                                    True])),
                           ('roles', array([1, 0, 0, 1, 0, 0, 1, 1, 0, 1])),
                           ('votes',
                            array([ 9.839711 ,  2.3241622,  2.1457787,  0.1778076, 10.52     ,
                                    9.540108 ,  6.323592 ,  3.98267  ,  1.2211397,  8.895571 ],
                                  dtype=float32))]))])

In [ ]:
class Agent(torch.nn.Module):
    def __init__(self, num_actions):
        super().__init__()

        self.network = torch.nn.Sequential(
            self._layer_init(torch.nn.Linear(obs_size, 64)),
            torch.nn.ReLU(),
            self._layer_init(torch.nn.Linear(64, 64)),
            torch.nn.ReLU(),
        )

        self.actor = self._layer_init(torch.nn.Linear(512, num_actions))
        self.critic = self._layer_init(torch.nn.Linear(512, 1))
    
    def _layer_init(self, layer, std=np.sqrt(2), bias_const=0.0):
        torch.nn.orthogonal_(layer.weight, std)
        torch.nn.constant_(layer.bias, bias_const)
        return layer